In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def init_pos(radius,numberOfParticles):
    to_close = True
    while(to_close):
        to_close = False
        r = np.random.uniform(0,radius, numberOfParticles)
        theta = np.random.uniform(0,2*np.pi, numberOfParticles)
        pos_x = r * np.cos(theta)
        pos_y = r * np.sin(theta)
        for i in range(numberOfParticles):
            for j in range(i+1,numberOfParticles):
                d = np.sqrt((pos_x[i] - pos_x[j])**2 + (pos_y[i] - pos_y[j])**2)
                if d < 1:
                    to_close = True
                    break
            if to_close:
                break
    return pos_x, pos_y

def random_velocities(numberOfParticles):
    angles =  np.random.uniform(-np.pi, np.pi,numberOfParticles)
    vx = np.cos(angles) / np.sqrt(numberOfParticles / 2)  
    vy = np.sin(angles) / np.sqrt(numberOfParticles / 2)  
    return vx, vy     

epsilon = 0.5
def billiard(numberOfParticles, numberOfIterations, radius, dt, KK):
    pos = np.zeros((2, numberOfParticles, numberOfIterations+1))
    pos[0,:,0], pos[1,:,0] = init_pos(radius,numberOfParticles)

    vx, vy = random_velocities(numberOfParticles)
    vx_list = np.zeros(numberOfIterations)

    Pot = np.zeros(numberOfIterations)
    kinetic = np.zeros(numberOfIterations)
    for i in range(numberOfIterations):
        vx_list[i] = vx[0]
        distance = np.sqrt(pos[0,:,i]**2 + pos[1,:,i]**2)
        acceleration = np.zeros((2, numberOfParticles))


        #test if we are outside the circle and calculate force from wall
        acceleration[0, distance > radius] = -KK * (distance[distance > radius] - radius) * pos[0, distance > radius, i] / distance[distance > radius]
        acceleration[1, distance > radius] = -KK * (distance[distance > radius] - radius) * pos[1, distance > radius, i] / distance[distance > radius]
        for j in range(numberOfParticles):
                for k in range(j+1, numberOfParticles):
                    d_jk = np.sqrt((pos[0,j,i] - pos[0,k,i])**2 + (pos[1,j,i] - pos[1,k,i])**2)
                    if d_jk < 1:
                        F = 12 * epsilon * (1/d_jk**7 - 1/d_jk**13)
                        
                        acceleration[0,j] += - F * (pos[0,j,i] - pos[0,k,i]) / d_jk 
                        acceleration[1,j] += - F * (pos[1,j,i] - pos[1,k,i]) / d_jk 
                        acceleration[0,k] += F * (pos[0,j,i] - pos[0,k,i]) / d_jk 
                        acceleration[1,k] += F * (pos[1,j,i] - pos[1,k,i]) / d_jk
                        
                        Pot[i] += epsilon * (1/d_jk**12 - 2/d_jk**6 + 1)

        #update positions
        pos[0, :, i+1] =  pos[0, :, i] + vx * dt + (acceleration[0, :] * dt**2)/2
        pos[1, :, i+1] =  pos[1, :, i] + vy * dt + (acceleration[1, :] * dt**2)/2
        
       
        distance = np.sqrt(pos[0,:,i+1]**2 + pos[1,:,i+1]**2)
        acceleration2 = np.zeros((2, numberOfParticles))

        #test if we are outside the circle and calculate force from wall
        acceleration2[0, distance > radius] = -KK * (distance[distance > radius] - radius) * pos[0, distance > radius, i+1] / distance[distance > radius]
        acceleration2[1, distance > radius] = -KK * (distance[distance > radius] - radius) * pos[1, distance > radius, i+1] / distance[distance > radius]
        for j in range(numberOfParticles):
                for k in range(j+1, numberOfParticles):
                    d_jk = np.sqrt((pos[0,j,i+1] - pos[0,k,i+1])**2 + (pos[1,j,i+1] - pos[1,k,i+1])**2)
                    if d_jk < 1:
                        F = 12 * epsilon * (1/d_jk**7 - 1/d_jk**13)
                        
                        acceleration2[0,j] += - F * (pos[0,j,i+1] - pos[0,k,i+1]) / d_jk 
                        acceleration2[1,j] += - F * (pos[1,j,i+1] - pos[1,k,i+1]) / d_jk 
                        acceleration2[0,k] += F * (pos[0,j,i+1] - pos[0,k,i+1]) / d_jk 
                        acceleration2[1,k] += F * (pos[1,j,i+1] - pos[1,k,i+1]) / d_jk
                        

        #calculate Energy
        Pot[i] += KK/2 * (distance - radius)**2 @ (distance>radius)
        kinetic[i] = np.sum(1/2 * (vx**2 + vy**2))

        #update velocities
        vx += (acceleration[0,:] + acceleration2[0,:])/2 * dt
        vy += (acceleration[1,:] + acceleration2[1,:])/2 * dt


    return pos, Pot, kinetic, vx_list